In [1]:
import numpy as np
import os
from pandarallel import pandarallel
pandarallel.initialize()
import sys
import pickle
from tqdm import tqdm
import pandas as pd

id_col = 'PanjivaRecordID'

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
DIR = 'us_import1'
SRC_DIR = os.path.join('./../generated_data_v1', DIR)
SAVE_DIR = os.path.join('./../generated_data_v1', DIR)

In [3]:
def get_domain_dims(DIR):
    with open(os.path.join('./../generated_data_v1', DIR, 'domain_dims.pkl'),'rb') as fh:
        domain_dims = pickle.load(fh)
    return domain_dims

In [4]:
def aux_gen(row, domain_dims, num_samples =1):
    nd = len(domain_dims)
    global id_col
     
    domains = list(domain_dims.keys())
    res = []
    pos = row.copy()
    del  pos[id_col]
    pos = pos.values
    for i in range(num_samples):
        
        row_copy = row.copy()
        del row_copy[id_col]
        num_pert = int(np.random.randint(nd//2, nd))
        pert_idx = np.random.choice(list(range(nd)),num_pert,replace=False)
        for _idx in pert_idx:
            dom = domains[_idx]
            row_copy[dom] = np.random.randint(0,domain_dims[dom])
        res.append(row_copy.values)
    neg = np.vstack(res)
    return (pos,neg)

In [8]:
def generate_neg_samples(DIR, num_samples = 5):
    global SRC_DIR

    domain_dims = get_domain_dims(DIR)
    data_df = pd.read_csv(os.path.join(SRC_DIR,'train_data.csv'), index_col=None).head(100)
    res = data_df.parallel_apply( aux_gen, axis=1, args=(domain_dims, num_samples,))
    pos = []
    neg = []
    for r in res:
        pos.append(r[0])
        neg.append(r[1])
    
    pos = np.vstack(pos)
    neg = np.vstack(neg).reshape( [pos.shape[0], num_samples, pos.shape[1]])
    print(pos.shape, neg.shape)
    return pos, neg
    # Save the data 
    np.save(
        SAVE_DIR + '/pos_data.npy',
        pos
    )
    np.save(
        SAVE_DIR + '/neg_data.npy',
        neg
    )
    return pos, neg

(array([ 222, 1300, 1223,  247,   37,   86,   16, 1652]),
 array([[ 560, 1300, 1294,  108,    5,   86,    0, 2347],
        [ 222, 1300, 1223,  159,   21,   86,   63, 2487],
        [  90, 1300, 1596,  247,    1,    2,   59,  504],
        [ 222, 1391, 1242,  247,   37,  116,   16, 2381],
        [ 264, 1300, 1285,  247,   37,   53,   64, 1652]]))